You are given a rows x cols matrix grid representing a field of cherries where grid[i][j] represents the number of cherries that you can collect from the (i, j) cell.

You have two robots that can collect cherries for you:

Robot #1 is located at the top-left corner (0, 0), and
Robot #2 is located at the top-right corner (0, cols - 1).
Return the maximum number of cherries collection using both robots by following the rules below:

From a cell (i, j), robots can move to cell (i + 1, j - 1), (i + 1, j), or (i + 1, j + 1).
When any robot passes through a cell, It picks up all cherries, and the cell becomes an empty cell.
When both robots stay in the same cell, only one takes the cherries.
Both robots cannot move outside of the grid at any moment.
Both robots should reach the bottom row in grid.

# Brute Force

In [1]:
import copy

In [2]:
#   TODO: should remove the shape thing and use a class maybe to store the value
def get_valid_actions(field_shape: tuple, current_loc:tuple, parent:bool) -> list[tuple]:
    """
    shapes are in (i, j) format
    """
    result = []
    if not parent:
        next_i = current_loc[0] + 1
    else:
        next_i = current_loc[0] - 1
    
    #   check i
    if next_i >= field_shape[0] or next_i < 0:
        return result
    
    for step in range(-1, 2):
        next_j = current_loc[1] + step
        #   check j
        if next_j < 0 or next_j >= field_shape[1]:
            continue
        result.append((next_i, next_j))
    
    return result


In [6]:
#   test
list_shape = (5,5)
locs = [(0,0), (0,4), (1,3)]
for x in locs:
    print(get_valid_actions(list_shape,x, True))

[]
[]
[(0, 2), (0, 3), (0, 4)]


In [ ]:
#   should start at a given loc
#   expand my next moves
#   store the path that im taking
list_shape = (5,5)
paths = []

def bfs(current_loc:tuple, path:list):
    path.append(current_loc)
    #   get next possible actions:
    next_actions = get_valid_actions(list_shape, current_loc)
    if not next_actions:
        #   termination
        paths.append(path)
        return
    
    for action in next_actions:
        #   important to use a new object and not the main reference
        bfs(action, copy.copy(path))

bfs((0,0),[])
for x in paths:
    print(x)

In [11]:
a = [1]

if not a:
    print("!!!")

#   DP Approach

In [22]:
#   single agent
def build_max_paths(board: list[list[int]]):
    paths = {}
    board_shape = (len(board), len(board[0]))
    max_rewards = [[0] * board_shape[1] for _ in range(board_shape[0])]
    frontier = []
    frontier.append((0,0))
    #   since we are using a queue we are going bfs which, since we need the parents data for the next layer
    #   while there is a node in the frontier
    while frontier:
        node = frontier.pop(0)
        
        #   get the parents
        parent_nodes = get_valid_actions(board_shape, node, parent=True)
        
        if parent_nodes:
            #   select the max parent and make the path
            max_parent_val = -1
            max_parent = None
            for parent in parent_nodes:
                if max_rewards[parent[0]][parent[1]] > max_parent_val:
                    max_parent = parent
                    max_parent_val = max_rewards[parent[0]][parent[1]]
        
            #   for i_j key format
            key = f"{max_parent[0]}_{max_parent[1]}"
            parent_path = paths[key]
            node_path = copy.copy(parent_path)
            node_path.append(node)
                
            node_max_reward = max_parent_val + board[node[0]][node[1]]
            max_rewards[node[0]][node[1]] = node_max_reward
            print(f"max reward added for {node}: {node_max_reward}")
        else:
            #   basically for the first level
            max_rewards[node[0]][node[1]] = board[node[0]][node[1]]
            print(f"max reward added for {node}: {board[node[0]][node[1]]}")
            node_path = [node]
        
        #   update paths and max values for the node coordinates
        
        node_key = f"{node[0]}_{node[1]}"
        paths[node_key] = node_path
        print(f"adding node path for key:{node_key} : {node_path}")
        #   get the children and add them to frontier
        children = get_valid_actions(board_shape, node, parent=False)
        frontier = frontier + children
        
    return paths, max_rewards
        

In [24]:
board = [[3,1,1],[2,5,1],[1,5,5],[2,1,1]]

paths, max_rewards = build_max_paths(board)

print(paths)
print(max_rewards)

max reward added for (0, 0): 3
adding node path for key:0_0 : [(0, 0)]
max reward added for (1, 0): 5
adding node path for key:1_0 : None
max reward added for (1, 1): 8
adding node path for key:1_1 : None


AttributeError: 'NoneType' object has no attribute 'append'